# Анализ БД для сервиса чтения книг по подписке с помощью SQL запросов

**ОГЛАВЛЕНИЕ:**

1. [Цель проекта](#start)

2. [Подключение к БД](#connection)
     
3. [Знакомство с данными](#info)
      
4. [Выполнение запросов](#query)

    4.1 [Запросы для получения основных данных о БД](#general) 
    
    4.2 [Запросы в рамках заданий](#tasks)
    
    4.2.1 [Количество книг с датой выпуска позже 1 января 2000 года](#year_release)
      
    4.2.2 [Подсчет средней оценки и количества обзоров для каждой книги](#avg_rating)
      
    4.2.3 [Определение издательства, выпустившего наибольшее число книг толще 50 страниц](#publisher)
      
    4.2.4 [Определение автора с самой высокой средней оценкой книг, имеющих 50 и более оценок](#author)
      
    4.2.5 [Подсчет среднего количества обзоров от пользователей, поставивших более 50 оценок](#avg_review)
      
    
5. [Выводы по проекту](#conclusion)

## Цель проекта<a id="start"></a>  

*Проанализировать базу данных с информацией о книгах, издательствах, авторах, а также пользовательских обзорах книг, используя SQL-запросы. Эти данные помогут сформулировать ценностное предложение для нового продукта - сервиса для чтения книг по подписке*

**Описание данных**

*Таблица `books`*

Содержит данные о книгах:

- `book_id` — идентификатор книги;
- `author_id` — идентификатор автора;
- `title` — название книги;
- `num_pages` — количество страниц;
- `publication_date` — дата публикации книги;
- `publisher_id` — идентификатор издателя.

*Таблица `authors`*

Содержит данные об авторах:

- `author_id` — идентификатор автора;
- `author` — имя автора.

*Таблица `publishers`*

Содержит данные об издательствах:

- `publisher_id` — идентификатор издательства;
- `publisher` — название издательства;

*Таблица `ratings`*

Содержит данные о пользовательских оценках книг:

- `rating_id` — идентификатор оценки;
- `book_id` — идентификатор книги;
- `username` — имя пользователя, оставившего оценку;
- `rating` — оценка книги.

*Таблица `reviews`*

Содержит данные о пользовательских обзорах на книги:

- `review_id` — идентификатор обзора;
- `book_id` — идентификатор книги;
- `username` — имя пользователя, написавшего обзор;
- `text` — текст обзора.

## Подключение к БД  <a id="connection"></a> 

In [1]:
# импортируем библиотеки
import pandas as pd
from sqlalchemy import create_engine

Выполним подключение к БД, используя параметры:

In [2]:
# устанавливаем параметры
db_config = {'user': 'praktikum_student', # имя пользователя
'pwd': 'Sdf4$2;d-d30pp', # пароль
'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',
'port': 6432, # порт подключения
'db': 'data-analyst-final-project-db'} # название базы данных
connection_string = 'postgresql://{}:{}@{}:{}/{}'.format(db_config['user'],
 db_config['pwd'],
 db_config['host'],
 db_config['port'],
 db_config['db'])
# сохраняем коннектор
engine = create_engine(connection_string, connect_args={'sslmode':'require'})


## Знакомство с данными <a id="info"></a>  

Выведем первые строки каждой таблицы, чтобы оптимизировать код, будем использовать функцию и цикл по названиям таблиц

In [3]:
def function_query(query):
    """Эта функция выполняет переданный на вход запрос,
    записывает его во внутреннюю переменную, выводит первые 5 строк результата на экран"""
# чтобы выполнить SQL-запрос, используем Pandas
    data_raw = pd.io.sql.read_sql(query, con = engine)
    display(data_raw)
    pd.options.display.max_colwidth = 150

In [4]:
#выводим первые строки таблиц в цикле
table_name = ['books', 'authors', 'ratings', 'reviews', 'publishers']
#пишем запрос в переменную
for t in table_name:

    query =  f'SELECT * FROM {t} LIMIT 5'
    #вызываем функцию
    function_query(query)

,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope...,322,2010-12-21,135
3,4,82,1491: New Revelations of the Americas Before C...,541,2006-10-10,309
4,5,125,1776,386,2006-07-04,268


,author_id,author
0,1,A.S. Byatt
1,2,Aesop/Laura Harris/Laura Gibbs
2,3,Agatha Christie
3,4,Alan Brennert
4,5,Alan Moore/David Lloyd


,rating_id,book_id,username,rating
0,1,1,ryanfranco,4
1,2,1,grantpatricia,2
2,3,1,brandtandrea,5
3,4,2,lorichen,3
4,5,2,mariokeller,2


,review_id,book_id,username,text
0,1,1,brandtandrea,Mention society tell send professor analysis. Over provide race technology continue these.
1,2,1,ryanfranco,Foot glass pretty audience hit themselves. Among admit investment argue security.
2,3,2,lorichen,Listen treat keep worry. Miss husband tax but person sport treatment industry. Kitchen decision deep the. Social party body the.
3,4,3,johnsonamanda,Finally month interesting blue could nature cultural bit. Prepare beat finish grow that smile teach. Dream me play near.
4,5,3,scotttamara,Nation purpose heavy give wait song will. List dinner another whole positive radio fast. Music staff many green.


,publisher_id,publisher
0,1,Ace
1,2,Ace Book
2,3,Ace Books
3,4,Ace Hardcover
4,5,Addison Wesley Publishing Company


In [5]:
# какие ещё таблицы доступны в базе
display(pd.io.sql.read_sql('''

SELECT *
FROM pg_catalog.pg_tables
WHERE schemaname != 'pg_catalog' AND
      schemaname != 'information_schema';

''', con = engine))


# смотрим на типы столбцов в интересующих таблицах
display(pd.io.sql.read_sql('''
SELECT 
    table_name, 
    column_name, 
    data_type, 
    is_nullable
FROM INFORMATION_SCHEMA.COLUMNS 
WHERE table_name IN ('books', 'authors', 'publishers', 'ratings', 'reviews');
''', con = engine))

,schemaname,tablename,tableowner,tablespace,hasindexes,hasrules,hastriggers,rowsecurity
0,public,orders,praktikum_admin,None,True,False,False,False
1,public,visits,praktikum_admin,None,True,False,False,False
2,public,advertisment_costs,praktikum_admin,None,True,False,False,False
3,public,authors,praktikum_admin,None,True,False,True,False
4,public,second,praktikum_student,None,False,False,False,False
5,public,second_b,praktikum_student,None,False,False,False,False
6,public,publishers,praktikum_admin,None,True,False,True,False
7,public,author,praktikum_student,None,True,False,False,False
8,public,reviews,praktikum_admin,None,True,False,True,False
9,public,ratings,praktikum_admin,None,True,False,True,False


,table_name,column_name,data_type,is_nullable
0,authors,author_id,integer,NO
1,authors,author,text,YES
2,publishers,publisher_id,integer,NO
3,publishers,publisher,text,YES
4,reviews,review_id,integer,NO
5,reviews,book_id,integer,YES
6,reviews,username,text,YES
7,reviews,text,text,YES
8,ratings,rating_id,integer,NO
9,ratings,book_id,integer,YES


## Выполнение запросов <a id="query"></a> 

### Запросы для получения основных данных о БД <a id="general"></a> 

Посчитаем, сколько всего книг содержится в БД:

In [6]:
query =  '''SELECT COUNT(*)
            FROM books
            
            '''
function_query(query)

,count
0,1000


In [7]:
#посчитаем количество авторов
query =  '''SELECT COUNT(DISTINCT author_id) 
            FROM books
            
            '''
function_query(query)

,count
0,636


In [8]:
#посчитаем количество издательств
query =  '''SELECT COUNT(DISTINCT publisher_id) 
            FROM books
            
            '''
function_query(query)

,count
0,340


In [9]:
#посчитаем минимальный, максимальный, средний рейтинг книги
query =  '''SELECT MIN(rating), ROUND(AVG(rating),2) AS avg, MAX(rating) 
            FROM ratings
            
            '''
function_query(query)

,min,avg,max
0,1,3.93,5


In [10]:
#посчитаем среднее количество обзоров по книге
query =  '''SELECT ROUND(AVG(c.cnt_review)) AS avg_reviews
            FROM
                (SELECT book_id, COUNT(review_id) AS cnt_review 
                FROM reviews
                GROUP BY book_id) AS c
            
            '''
function_query(query)

,avg_reviews
0,3.0


In [11]:
#посчитаем, сколько книг имеет отзывы
query =  '''
            SELECT COUNT(DISTINCT book_id) AS cnt_books_in_review
                        FROM reviews
                     
            '''
function_query(query)

,cnt_books_in_review
0,994


In [12]:
#посчитаем, все ли книги оценены
query =  '''
            SELECT COUNT(DISTINCT book_id) AS cnt_books_in_rating
                        FROM ratings
                     
            '''
function_query(query)

,cnt_books_in_rating
0,1000


In [13]:
#найдем самого активного пользователя по ревью
query =  '''SELECT username, COUNT(review_id) AS cnt_rev
            FROM reviews
            GROUP BY username
            ORDER BY cnt_rev DESC
            LIMIT 1
        '''
function_query(query)

,username,cnt_rev
0,susan85,29


In [14]:
#найдем самого активного пользователя по оценкам
query =  '''SELECT username, COUNT(rating_id) AS cnt_rat
            FROM ratings
            GROUP BY username
            ORDER BY cnt_rat DESC
            LIMIT 1
        '''
function_query(query)

,username,cnt_rat
0,paul88,56


In [15]:
#найдем ТОП-3 самых крупных издательств
query =  '''SELECT p.publisher, b.cnt_books
            FROM publishers AS p
            INNER JOIN (
                SELECT publisher_id, COUNT(book_id) AS cnt_books
                FROM books 
    
                GROUP BY publisher_id
                ORDER BY cnt_books DESC
                LIMIT 3) AS b ON p.publisher_id = b.publisher_id
        '''
function_query(query)


,publisher,cnt_books
0,Penguin Books,42
1,Vintage,31
2,Grand Central Publishing,25


In [16]:
#найдем ТОП-5 авторов, которым принадлежит максимальное количество книг в БД
query =  '''SELECT a.author, b.cnt_books
            FROM authors AS a
            INNER JOIN (
                SELECT author_id, COUNT(book_id) AS cnt_books
                FROM books 
    
                GROUP BY author_id
                ORDER BY cnt_books DESC
                LIMIT 5) AS b ON a.author_id = b.author_id
        '''
function_query(query)


,author,cnt_books
0,Terry Pratchett,15
1,Laurell K. Hamilton,14
2,John Grisham,14
3,Stephen King,13
4,James Patterson,12


Посчитаем, сколько книг в БД имеют максимальный средний рейтинг = 5:

In [17]:
query =  '''SELECT COUNT(b.book_id) as cnt_book
            FROM (
                SELECT book_id, AVG(rating) as avg_rat
                FROM ratings
                GROUP BY book_id
                ORDER BY avg_rat DESC
                ) AS b
            WHERE b.avg_rat = 5
        '''
function_query(query)            

,cnt_book
0,44


Найдем ТОП-5 книг с максимальным количеством отзывов:

In [18]:
query =  '''SELECT b.title, a.cnt_rev
            FROM books AS b
            INNER JOIN (
                SELECT book_id, COUNT(review_id) AS cnt_rev
                FROM reviews
                GROUP BY book_id
                ORDER BY cnt_rev DESC
                LIMIT 5) AS a ON a.book_id = b.book_id
        '''
function_query(query)


,title,cnt_rev
0,Twilight (Twilight #1),7
1,The Hobbit or There and Back Again,6
2,The Curious Incident of the Dog in the Night-Time,6
3,Water for Elephants,6
4,The Da Vinci Code (Robert Langdon #2),6


In [19]:
#посмотрим даты выхода книг
query =  '''SELECT MIN(publication_date), MAX(publication_date)
            FROM books
        '''
function_query(query)

,min,max
0,1952-12-01,2020-03-31


In [20]:
#посмотрим, сколько пользователей в БД
query =  '''SELECT COUNT(DISTINCT username) AS cnt_users

            FROM ratings
        '''
function_query(query)

,cnt_users
0,160


In [21]:
#посмотрим среднее количество оценок на пользователя
query =  '''SELECT ROUND(AVG(c.cnt_scores_per_user)) AS avg_scores
            FROM (SELECT username, COUNT(rating_id) AS cnt_scores_per_user
                FROM ratings
                GROUP BY username) AS c
        '''
function_query(query)

,avg_scores
0,40.0


In [22]:
#посмотрим среднее количество отзывов на пользователя
query =  '''SELECT ROUND(AVG(c.cnt_review_per_user)) AS avg_reviews
            FROM (SELECT username, COUNT(review_id) AS cnt_review_per_user
                FROM reviews
                GROUP BY username) AS c
        '''
function_query(query)

,avg_reviews
0,17.0


Основная информация о БД:
- В БД имеется информация о 1000 книг. 
- Эти книги созданы 636 авторами, изданы 340 издательствами. 
- Самая старая книга в БД вышла 1 декабря 1952 года, самая новая - 31 марта 2020
- Книги оценены по рейтингу от 1 до 5, средний рейтинг книг составляет 3.93
- Среднее количество отзывов на книгу - 3
- 994 книги имеют отзывы, оценку - все книги из БД
- В БД зарегистрировано 160 пользователей, они проставляют в среднем 40 оценок, оставляют в среднем 17 отзывов
- Самый активный пользователь по составлению отзывов - susan85, этот пользователь оставил 29 отзывов.
- Самый активный пользователь по оценкам - paul88, он дал оценку 56 раз.
- Максимальное количество книг выпустило издательство Penguin Books - 42 книги. 
- Автору Terry Pratchett принадлежит 15 книг в БД, это наибольшее значение.
- Книга "Twilight (Twilight #1)" имеет 7 отзывов, максимум по базе. 
- 44 книги в БД имеют средний рейтинг, равный максимальной оценке в 5.

### Запросы в рамках заданий  <a id="tasks"></a> 

#### Количество книг с датой выпуска позже 1 января 2000 года<a id="year_release"></a>

Посчитаем сколько книг вышло после 1 января 2000 года

In [23]:
query =  '''--считаем кол-во книг с выпуском после 1 января 2000 года
            SELECT COUNT(book_id) AS count_books
            FROM books
            WHERE publication_date > '2000-01-01'
            '''
function_query(query)

,count_books
0,819


Более половины книг вышло после 1 января 2000 года. 

#### Подсчет средней оценки и количества обзоров для каждой книги<a id="avg_rating"></a>

Для каждой книги посчитаем количество обзоров и среднюю оценку:

In [24]:
query =  '''--считаем кол-во обзоров и среднюю оценку для каждой книги
        SELECT b.book_id,
            b.title,
           ROUND(i.avg_rating, 2) AS avg_rating,
           c.total_reviews 
        FROM books AS b
         LEFT OUTER JOIN 
          (SELECT book_id,
              AVG(rating) AS avg_rating
           FROM ratings
           GROUP BY book_id
           
           ) AS i ON i.book_id = b.book_id
        LEFT OUTER JOIN
          (SELECT book_id,
              COUNT(review_id) AS total_reviews
           FROM reviews
           GROUP BY book_id
           ORDER BY total_reviews DESC) AS c ON i.book_id = c.book_id
           ORDER BY avg_rating DESC
           
        '''
function_query(query)

,book_id,title,avg_rating,total_reviews
0,518,Pop Goes the Weasel (Alex Cross #5),5.00,2.0
1,732,The Ghost Map: The Story of London's Most Terrifying Epidemic—and How It Changed Science Cities and the Modern World,5.00,2.0
2,347,In the Hand of the Goddess (Song of the Lioness #2),5.00,2.0
3,610,Tai-Pan (Asian Saga #2),5.00,2.0
4,330,How to Be a Domestic Goddess: Baking and the Art of Comfort Cooking,5.00,1.0
...,...,...,...,...
995,915,The World Is Flat: A Brief History of the Twenty-first Century,2.25,3.0
996,371,Junky,2.00,2.0
997,316,His Excellency: George Washington,2.00,2.0
998,202,Drowning Ruth,2.00,3.0


In [25]:
query =  '''--считаем кол-во обзоров и среднюю оценку для каждой книги
        WITH a AS (
                SELECT book_id,
                  AVG(rating) AS avg_rating
               FROM ratings
               GROUP BY book_id
           
              ),
              b AS (
              SELECT book_id,
                  COUNT(review_id) AS total_reviews
               FROM reviews
               GROUP BY book_id
           
               )
              SELECT c.book_id,
                c.title, a.avg_rating,
               COALESCE(b.total_reviews, 0) AS total_rev
               FROM books AS c
               LEFT OUTER JOIN a ON a.book_id = c.book_id
               LEFT OUTER JOIN b ON b.book_id = c.book_id
               ORDER BY a.avg_rating DESC,  total_rev DESC
        
        
        '''
function_query(query) 

,book_id,title,avg_rating,total_rev
0,17,A Dirty Job (Grim Reaper #1),5.00,4
1,444,Moneyball: The Art of Winning an Unfair Game,5.00,3
2,553,School's Out—Forever (Maximum Ride #2),5.00,3
3,459,My Name Is Asher Lev,5.00,2
4,972,Wherever You Go There You Are: Mindfulness Meditation in Everyday Life,5.00,2
...,...,...,...,...
995,915,The World Is Flat: A Brief History of the Twenty-first Century,2.25,3
996,202,Drowning Ruth,2.00,3
997,371,Junky,2.00,2
998,316,His Excellency: George Washington,2.00,2


3 книги в БД имеют максимальный средний рейтинг = 5 и количество отзывов = 4 и 3. Это "A Dirty Job (Grim Reaper #1)", "School's Out—Forever (Maximum Ride #2)", "Moneyball: The Art of Winning an Unfair Game"

#### Определение издательства, выпустившего наибольшее число книг толще 50 страниц<a id="publisher"></a>

Определим издательство, которое выпустило наибольшее число книг толще 50 страниц — так мы исключим из анализа брошюры

In [26]:

query =  '''--ищем издат-во, выпустившее наибольшее число книг толще 50 страниц
            SELECT p.publisher, s.cnt_books
            FROM publishers AS p
            INNER JOIN
                (SELECT publisher_id, COUNT(book_id) AS cnt_books
                FROM books
                WHERE num_pages > 50
                GROUP BY publisher_id
                ORDER BY cnt_books DESC
                LIMIT 1) AS s ON s.publisher_id = p.publisher_id
            '''
function_query(query)

,publisher,cnt_books
0,Penguin Books,42


Даже с учетом исключения брошюр лидер остался тот же - Penguin Books	

#### Определение автора с самой высокой средней оценкой книг, имеющих 50 и более оценок<a id="author"></a>

Определим автора с самой высокой средней оценкой книг, учитывая только книги с 50 и более оценками

In [27]:
query = ''' --ищем авторов с самой высокой средней оценкой книг, получивших 50 и более оценок
            WITH c AS (
                    SELECT book_id, AVG(rating) as avg_rating
                    FROM ratings
                    GROUP BY book_id
                    HAVING COUNT(rating_id) >= 50
                    ORDER BY avg_rating DESC
                    --LIMIT 50
                )
            SELECT a.author, AVG(c.avg_rating) AS avg_rat
            FROM authors AS a
            INNER JOIN books AS b ON b.author_id = a.author_id
            INNER JOIN c ON c.book_id = b.book_id
            GROUP BY a.author
            ORDER BY avg_rat DESC
                    
        '''
function_query(query)

,author,avg_rat
0,J.K. Rowling/Mary GrandPré,4.283844
1,Markus Zusak/Cao Xuân Việt Khương,4.264151
2,J.R.R. Tolkien,4.258446
3,Louisa May Alcott,4.192308
4,Rick Riordan,4.080645
5,William Golding,3.901408
6,J.D. Salinger,3.825581
7,Paulo Coelho/Alan R. Clarke/Özdemir İnce,3.789474
8,William Shakespeare/Paul Werstine/Barbara A. Mowat,3.787879
9,Dan Brown,3.754540


Критериям запроса удовлетворяют 14 авторов БД. Но лидер среди них - J.K. Rowling, у нее самый высокий средний рейтинг книг.

Найдем, какая книга этого автора больше всех понравилась читателям (самый высокий средний рейтинг и 50 и более оценок):

In [29]:
query = ''' --ищем самую популярную книгу лучшего автора
            SELECT a.author, b.book_id, b.title
            FROM authors AS a
            INNER JOIN books AS b ON b.author_id = a.author_id
            WHERE b.book_id IN
                (
                    SELECT book_id
                    FROM ratings
                    GROUP BY book_id
                    HAVING COUNT(rating_id) >= 50
                    ORDER BY AVG(rating) DESC
                    LIMIT 1
                    )
        '''
function_query(query)

,author,book_id,title
0,J.K. Rowling/Mary GrandPré,302,Harry Potter and the Prisoner of Azkaban (Harry Potter #3)


"Гарри Поттер и узник Азкабана" - фаворит читателей

#### Подсчет среднего количества обзоров от пользователей, поставивших более 50 оценок<a id="avg_review"></a>

Посчитаем среднее количество обзоров от пользователей, которые поставили больше 50 оценок


In [30]:
query = '''
        --считаем среднее количество обзоров пользователей, поставивших более 50 оценок
        SELECT ROUND(AVG(c.cnt_review),2) AS avg_reviews
        FROM (
            WITH a AS (
                SELECT username, COUNT(review_id) AS cnt_review
                FROM reviews
                GROUP BY username),
            b AS
                (SELECT username
                FROM ratings
                GROUP BY username
                HAVING COUNT(rating_id) > 50
                )
            SELECT b.username, a.cnt_review
            FROM b
            LEFT OUTER JOIN a ON a.username = b.username
            ORDER BY a.cnt_review DESC
        ) AS c'''

function_query(query)            

,avg_reviews
0,24.33


Активные пользователи оставили в среднем 24 отзыва.

Выведем пользователей, которые поставили более 50 оценок, и количество обзоров, которое они составили:

In [31]:
query = '''WITH a AS (
                SELECT username, COUNT(review_id) AS cnt_review
                FROM reviews
                GROUP BY username),
            b AS
                (SELECT username
                FROM ratings
                GROUP BY username
                HAVING COUNT(rating_id) > 50
                )
            SELECT b.username, a.cnt_review
            FROM b
            LEFT OUTER JOIN a ON a.username = b.username
            ORDER BY a.cnt_review DESC
        '''
function_query(query)

,username,cnt_review
0,sfitzgerald,28
1,martinadam,27
2,richard89,26
3,jennifermiller,25
4,paul88,22
5,xdavis,18


В базе имеется 6 самых активных пользователей, как в части проставления оценок для книг, так и в части написания отзывов. Лидер - пользователь с ником sfitzgerald. Он поставил более 50  оценок и составил 28 отзывов.

## Выводы по проекту <a id="conclusion"></a> 

<div class="alert alert-info">  
В рамках данного проекта средствами Python было произведено подключение к БД, содержащей информацию о книгах, авторах, издательствах, рейтингах и отзывов пользователей.
    
Получение основной информации о БД, а также выполнение заданий по проекту было осуществлено с помощью SQL-запросов.
Получены следующие результаты:
    
- В БД имеется информация о 1000 книг. 
    
- Эти книги созданы 636 авторами, изданы 340 издательствами. 
    
- Самая старая книга в БД вышла 1 декабря 1952 года, самая новая - 31 марта 2020
    
- Книги оценены по рейтингу от 1 до 5, средний рейтинг книг составляет 3.93
    
- Среднее количество отзывов на книгу - 3
    
- 994 книги имеют отзывы, оценку - все книги из БД
    
- В БД зарегистрировано 160 пользователей, они проставляют в среднем 40 оценок, оставляют в среднем 17 отзывов    
    
- Самый активный пользователь по составлению отзывов - susan85, этот пользователь оставил 29 отзывов.
    
- Самый активный пользователь по оценкам - paul88, он дал оценку 56 раз.
    
- Максимальное количество книг выпустило издательство Penguin Books - 42 книги. 
    
- Автору Terry Pratchett принадлежит 15 книг в БД, это наибольшее значение.
    
- Книга "Twilight (Twilight #1)" имеет 7 отзывов, максимум по базе. 
    
- 44 книги в БД имеют средний рейтинг, равный максимальной оценке в 5.

- Более половины книг вышло после 1 января 2000 года.
    
- Penguin Books издало максимальное количество книг толще 50 страниц

- Автор с самой высокой средней оценкой книг (учитывая только книги с 50 и более оценками) - J.K. Rowling/Mary GrandPré
    
- Ее книга "Гарри Поттер и узник Азкабана" - фаворит читателей
    
- Активные пользователи, которые поставили более 50 оценок, оставили в среднем 24 отзыва
    
- Пользователь с ником sfitzgerald поставил более 50 оценок и составил 28 отзывов
    
    
    
</div>    